In [1]:
import os
import sys
import types
sys.path.insert(0, os.path.abspath('..'))

import itertools

from autocnet.examples import get_path
from autocnet.graph.network import CandidateGraph
from autocnet.graph.edge import Edge
from autocnet.matcher.matcher import FlannMatcher
from autocnet.matcher.suppression_funcs import distance, error
from autocnet.utils.utils import normalize_vector
from autocnet.fileio.io_controlnetwork import to_isis
from autocnet.matcher.subpixel import clip_roi
from autocnet.matcher.matcher import pattern_match

from IPython.display import display
import cv2

import pandas as pd

%pylab inline
import line_profiler
%load_ext line_profiler

Populating the interactive namespace from numpy and matplotlib


## Generate a 2 image adjacenecy graph

In [2]:
#Point to the adjacency Graph
adjacency = get_path('three_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

#Apply SIFT to extract features
cg.extract_features(method='sift')

#Match
cg.match_features()
    
#Apply outlier detection
cg.symmetry_checks()
cg.ratio_checks()


#Compute a homography and apply RANSAC
cg.compute_fundamental_matrices(clean_keys=['symmetry', 'ratio'])
cg.suppress(clean_keys=['fundamental'], suppression_func=error, k=30)


/Users/jlaura/github/autocnet/autocnet/matcher/outlier_detector.py:256: UserWarning: Unable to optimally solve.  Returning with 34 points
  warnings.warn('Unable to optimally solve.  Returning with {} points'.format(len(result)))


In [ ]:
cnet = cg.get_cnet(clean_keys=['suppression'])

In [ ]:
to_isis('out.cnet', cnet, mode='wb')

In [ ]:
figsize(10,10)
cg[1][2].plot(clean_keys=['suppression'], line_kwargs={'linewidth':0})
show()
cg[0][1].plot(clean_keys=['suppression'], line_kwargs={'linewidth':0})
show()
cg[0][2].plot(clean_keys=['suppression'], line_kwargs={'linewidth':0})

In [ ]:
cg.compute_triangular_cycles()

In [ ]:
x = set([1,2,3])
x.difference(set([2]))

In [ ]:
list(itertools.combinations(*cg.compute_triangular_cycles(), 2))

In [ ]:
print(cg[0][1].matches)

In [8]:
e = cg.edge[0][1]
F = e.fundamental_matrix

In [6]:
from scipy.optimize import leastsq

In [29]:
def cost_function():
    pass

def triangulate(x, x1, p, p1):
    """
    Triangulate sets of correspondences using estimated cameras
    
    Parameters
    ----------
    x : ndarray
        (n,2) x,y coordinates or (n,3) x,y homogeneous coordinates
    
    x1 : ndarray
        (n,2) x,y coordinates or (n,3) x,y homogeneous coordinates
        
    p : ndarray
        (3,4) camera matrix
        
    p1 : ndarray
         (3,4 camera matrix)
    
    Returns
    -------
    """
    
    
u, s, v = np.linalg.svd(F)
e = u[:,2] # Where entries are?
e1 = np.array([[0, e[2], -e[1]],
               [-e[2], 0, e[0]],
               [e[1], -e[0], 0]])
p1 = np.zeros((3,4))
p1[:,:3] = (e1 * F)
p1[:,3] = e
p = np.eye(3,4)

valid_coords = F.mas
pgs = lstsq()

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]] 
 [[  0.00000000e+00   1.07632759e-08   1.21069353e-04  -9.99799731e-01]
 [  1.48693064e-08   0.00000000e+00   3.11836738e-01   2.00115329e-02]
 [  3.15128726e-04   2.88985699e-01   0.00000000e+00   1.92813801e-04]]


In [ ]:
F.error.describe()

In [ ]:
print(e.matches)

The code below defaults to using the fundamental mask, but a better mask is probably the suppression base mask.

In [ ]:
def show_base(edge, ax=None, clean_keys=[], image_space=100,
              scatter_kwargs={}, line_kwargs={}, image_kwargs={}):
    """
    Plot the correspondences for a given edge

    Parameters
    ----------
    edge : object
           A graph edge object

    ax : object
         A MatPlotLIb axes object

    clean_keys : list
                 of strings of masking array names to apply

    image_space : int
                  The number of pixels to insert between the images

    scatter_kwargs : dict
                     of MatPlotLib arguments to be applied to the scatter plots

    line_kwargs : dict
                  of MatPlotLib arguments to be applied to the lines connecting matches

    image_kwargs : dict
                   of MatPlotLib arguments to be applied to the image rendering

    Returns
    -------
    ax : object
         A MatPlotLib axes object.  Either the argument passed in
         or a new object
    """

    if ax is None:
        ax = plt.gca()

    # Plot setup
    ax.set_title('Matching: {} to {}'.format(edge.source.image_name,
                                             edge.destination.image_name))
    ax.margins(tight=True)
    ax.axis('off')

    # Image plotting
    source_array = edge.source.get_array()
    destination_array = edge.destination.get_array()

    s_shape = source_array.shape
    d_shape = destination_array.shape

    y = max(s_shape[0], d_shape[0])
    x = s_shape[1] + d_shape[1] + image_space
    composite = np.zeros((y, x))

    composite[:, :s_shape[1]] = source_array
    composite[:, s_shape[1] + image_space:] = destination_array

    if 'cmap' in image_kwargs:
        cmap = image_kwargs['cmap']
    else:
        cmap = 'Greys'

    ax.imshow(composite, cmap=cmap)

In [ ]:
figsize(20,20)

plotfig = False

def deepen(nodes, threshold=4, base_mask='fundamental'):
    
    def _process(a,b, threshold, base_mask):        
        # Maintain node/edge ordering
        if a[0] > a[1]:
            a.reverse()
    
        if b[0] > b[1]:
            b.reverse()
    
        ab = cg[a[0]][a[1]]
        bc = cg.edge[b[0]][b[1]]

        if not 'depth' in bc.masks.columns:
            bc.masks['depth'] = bc.masks[base_mask]

        
        # F and masks from the edge we are deepening (target)
        F = bc.fundamental_matrix
        
        # Matched points from the edge we are using to deepen (source)
        base_mask = ab.masks[base_mask]
        match_idx = ab.matches[base_mask]

        max_distance = match_idx['distance'].max() * 1.5
        max_distance = 200
        # Edge AB points that we want to search for in BC
        ab_x = ab.source.get_keypoint_coordinates(index=match_idx['source_idx'], homogeneous=True)
        
        # All edge BC points that we want to try and match
        bc_x = bc.destination.get_keypoint_coordinates(homogeneous=True).values
        
        # Compute and normalize the epipolar lines
        l_norm = normalize_vector(ab_x.dot(F.T))

        # Project every AB point to a BC epipolar line and find all BC points within threshold of said line
        for i, (j, row) in enumerate(ab_x.iterrows()):
            # j = index back into the network data structure for matches / keypoints
            # i = the position index into the epipolar lines
            if plotfig:
                show_base(bc)
                plot(row['x'], row['y'], 'ro')
            
            # Get the AB descriptor to use as the reference point
            source_descriptor = ab.source.descriptors[j]

            # Grab the correct epipolar line and compute the distance between all BC points and that said line
            epipolar_line = l_norm[i]  
            dist = np.abs(epipolar_line.dot(bc_x.T))
            # Find all points within threshold of the current epipolar line
            ids = np.where(dist <= threshold)[0]
            
            # Flags for finding the best match
            flag_distance = math.inf
            # Setup as a None array so that we can perform an if.any() check later
            match = np.array([None])
            
            # Step through all of the candidate matches
            colors = []
            x = []
            y = []
            for k, v in zip(ids, dist[ids]):
                didx = bc_x[k]
                destination_descriptor = bc.destination.descriptors[k]
                descriptor_distance = cv2.norm(source_descriptor, destination_descriptor)
                colors.append(descriptor_distance)
                x.append(didx[0] + 1112)
                y.append(didx[1])
                #if plotfig:
                    #plot(didx[0]+ 100 + 1012, didx[1], 'bo', alpha=0.5, markersize=20)
                
                if descriptor_distance < flag_distance:# and descriptor_distance < max_distance:
                    flag_distance = descriptor_distance
                    
                    match = didx
                    iden = k
                    
            if match.any():
                if plotfig:
                    plot(match[0] + 100 + 1012, match[1], 'ro')

                potential_match = bc.matches[(bc.matches['source_idx'] == row.name) &
                                             (bc.matches['destination_idx'] == iden)]
                
                if len(potential_match) > 0:
                    idx = potential_match.iloc[0].name
                    if bc.masks['depth'].iloc[idx] == False:
                        print('Already included')
                    else:
                        print('Adding')
                        bc.masks['depth'].iloc[idx] = True
                else:
                    print('NEW MATCH....CRAP')
                #print(bc.matches[bc.matches['destination_idx'] == match.name])
            if plotfig:
                sc = scatter(x, y, c=colors, s=200, alpha=0.25)
                colorbar(sc)
                show() 


    
    subset_nodes = set(nodes)
    for n in nodes:
        sn = set([n])
        o = list(subset_nodes.difference(sn))
        
        a = [n, o[0]]
        b = [n, o[1]]
        
        _process(a,b,threshold, base_mask)
        _process(b,a,threshold, base_mask)
        


#%lprun -f deepen deepen(cg.compute_triangular_cycles()[0], base_mask='suppression')
deepen(cg.compute_triangular_cycles()[0], base_mask='suppression')

In [ ]:
import cv2
figsize(10,10)

plotfig = False

def deepen(nodes, threshold=1.5, base_mask='fundamental'):
    
    def _process(a,b, base_mask):        
        # Maintain node/edge ordering
        if a[0] > a[1]:
            a.reverse()
    
        if b[0] > b[1]:
            b.reverse()
    
        ab = cg[a[0]][a[1]]
        bc = cg.edge[b[0]][b[1]]

        ab_footprint = ab.source.geodata.footprint
        bc_footprint = bc.destination.geodata.footprint
        
        #print(ab_footprint, bc_footprint)

        
        if not 'depth' in bc.masks.columns:
            bc.masks['depth'] = bc.masks[base_mask]

        
        # F and masks from the edge we are deepening (target)
        F = bc.fundamental_matrix
        
        # Matched points from the edge we are using to deepen (source)
        base_mask = ab.masks[base_mask]
        match_idx = ab.matches[base_mask]

        # Edge AB points that we want to search for in BC
        ab_x = ab.source.get_keypoint_coordinates(index=match_idx['source_idx'], homogeneous=True)
        
        # All edge BC points that we want to try and match
        bc_x = bc.destination.get_keypoint_coordinates(homogeneous=True).values
        
        # Compute and normalize the epipolar lines
        l_norm = normalize_vector(ab_x.dot(F.T))

        # Project every AB point to a BC epipolar line and find all BC points within threshold of said line
        for i, (j, row) in enumerate(ab_x.iterrows()):
            # j = index back into the network data structure for matches / keypoints
            # i = the position index into the epipolar lines
            if plotfig:
                show_base(bc)
                plot(row['x'], row['y'], 'ro')
            
            # Get the AB descriptor to use as the reference point
            # source_descriptor = ab.source.descriptors[j]
            # Get the AB search
            template = clip_roi(ab.source.geodata, (row['x'], row['y']), 15)
            
            # Grab the correct epipolar line and compute the distance between all BC points and that said line
            epipolar_line = l_norm[i]  
            dist = np.abs(epipolar_line.dot(bc_x.T))
            # Find all points within threshold of the current epipolar line
            ids = np.where(dist <= threshold)[0]
            
            # Flags for finding the best match
            flag_distance = 0
            # Setup as a None array so that we can perform an if.any() check later
            match = np.array([None])
            
            # Step through all of the candidate matches
            for k, v in zip(ids, dist[ids]):
                didx = bc_x[k]
                image = clip_roi(bc.destination.geodata, didx[:2], 111)
                x, y, max_corr = pattern_match(template, image, upsampling=1)
                #print(x, y, max_corr)

                if plotfig:
                    plot(didx[0]+ 100 + 1012, didx[1], 'bo', alpha=0.5, markersize=20)
                    
                # The maximum move
                magnitude = True
                if (abs(x) >= 2 and abs(y) >= 2) or (abs(x) >= 4 or abs(y) >= 4):
                    magnitude = False
                    
                if max_corr > flag_distance and max_corr >= 0.9 and magnitude: # and descriptor_distance < max_distance:
                    flag_distance = max_corr
                    match = didx
                    iden = k
                    nx, ny = x, y
                    
            if match.any():
                if plotfig:
                    plot(match[0] + 100 + 1012, match[1], 'ro')

                potential_match = bc.matches[(bc.matches['source_idx'] == row.name) &
                                             (bc.matches['destination_idx'] == iden)]

                if len(potential_match) > 0:
                    idx = potential_match.iloc[0].name
                    if bc.masks['depth'].iloc[idx] == True:
                        print('ALREADY INCLUDED')
                    else:
                        print('INCLUDING')
                        bc.masks['depth'].iloc[idx] = True
                else:
                    print('NEW', flag_distance, nx, ny)
                    plot(row['x'], row['y'], 'ro')
                    plot(match[0] + 100 + 1012, match[1], 'ro')
                    show_base(bc)
                    show()
                #print(bc.matches[bc.matches['destination_idx'] == match.name])
            if plotfig:
                #sc = scatter(x, y, c=colors, s=200, alpha=0.25)
                #colorbar(sc)
                show() 


    
    subset_nodes = set(nodes)
    for n in nodes:
        sn = set([n])
        o = list(subset_nodes.difference(sn))
        
        a = [n, o[0]]
        b = [n, o[1]]
        
        _process(a,b, base_mask)
        _process(b,a, base_mask)
        


#%lprun -f deepen deepen(cg.compute_triangular_cycles()[0], base_mask='suppression')
deepen(cg.compute_triangular_cycles()[0], base_mask='suppression')

In [ ]:
# Remove the depth column for testing.
for s, d, e, in cg.edges_iter(data=True):
    e.masks.drop('depth', 1)

In [ ]:
figsize(10,10)
import pandas as pd
from scipy.spatial.distance import hamming


threshold = 4

e = cg[0][1]
F = e.fundamental_matrix
match_idx = e.matches[F.mask]
x = e.source.get_keypoint_coordinates(index=match_idx['source_idx'], homogeneous=True)
x1 = e.destination.get_keypoint_coordinates(homogeneous=True).values



"""
Normalization allows for the dot product of a 
point to the normalized line returns the distance.
"""
def normalize_vector(line):
    """
    Normalize the vector.
    """
    if isinstance(line, pd.DataFrame):
        line = line.values
    n = np.sqrt(line[:,0]**2 + line[:,1]**2).reshape(-1,1)
    line /= n
    return line

#Normalize the vector
l_norm = normalize_vector(x.dot(F.T))
for i, (j, row) in enumerate(x.iterrows()):
    source_descriptor = e.source.descriptors[j]
    epipolar_line = l_norm[i]    
    dist = np.abs(epipolar_line.dot(x1.T))
    candidates = dist.argsort()
    dists = dist[candidates]
    # Plot the base
    show_base(e)
    plot(row['x'], row['y'], 'ro')
    flag_distance = math.inf
    match = None
    for k in dists[dists < threshold]:
        didx = x1[candidates[k]]
        print(didx)
        destination_descriptor = e.destination.descriptors[candidates[k]]

        destination_descriptor = e.destination.descriptors[candidates[k]]
        descriptor_distance = cv2.norm(source_descriptor, destination_descriptor, cv2.NORM_L2)
        plot(didx[0]+ 100 + 1012, didx[1], 'bo')
        if descriptor_distance < flag_distance:
            flag_distance = descriptor_distance
            match = didx
    try:
        plot(match[0] + 100 + 1012, match[1], 'ro')
    except:pass
    show()
    if i == 10:
        break


As above, except, try rto use the fundamental matrix from edge `[0][2]` to increase depth on `[0][1]`.

In [ ]:
THRESHOLD = 4

e = cg[0][2]
F = e.fundamental_matrix
match_idx = e.matches[e.fundamental_matrix.mask]

x = e.source.get_keypoint_coordinates(index=match_idx['source_idx'], homogeneous=True)
x1 = e.destination.get_keypoint_coordinates(index=match_idx['destination_idx'], homogeneous=True)

#Normalize the vector
l_norms = normalize_vector(x.dot(F.T))

F_error = np.abs(np.sum(l_norms * x1, axis=1))
print(F_error.describe())
#print(dists)


    
    

In [ ]:
import networkx as nx

In [ ]:
# from eberly http://www.geometrictools.com/Documentation/MinimalCycleBasis.pdf
coords = {}
coords[0] = 1,8
coords[1] = 1,7
coords[2] = 4,7
coords[3] = 0,4
coords[4] = 5,4
coords[5] = 3,5
coords[6] = 2, 4.5
coords[7] = 6.5, 9
coords[8] = 6.2, 5
coords[9] = 5.5,3
coords[10] = 7,3
coords[11] = 7.5, 7.25
coords[12] = 8,4
coords[13] = 11.5, 7.25
coords[14] = 9, 1
coords[15] = 11, 3
coords[16] = 12, 2
coords[17] = 12, 5
coords[18] = 13.5, 6
coords[19] = 14, 7.25
coords[20] = 16, 4
coords[21] = 18, 8.5
coords[22] = 16, 1
coords[23] = 21, 1
coords[24] = 21, 4
coords[25] = 18, 3.5
coords[26] = 17, 2
coords[27] = 19, 2


vertices = {}
for v in range(28):
    vertices[v] = []
    
vertices[1] = [2,3]
vertices[2] = [1,4,7]
vertices[3] = [1,4]
vertices[4] = [2,3,5]
vertices[5] = [4,6]
vertices[6] = [5]
vertices[7] = [2,11]
vertices[8] = [9,10]
vertices[9] = [8,10]
vertices[10] = [8,9]
vertices[11] = [7,12,13]
vertices[12] = [11,13,20]
vertices[13] = [11,12,18]
vertices[14] = [15]
vertices[15] = [14, 16]
vertices[16] = [15]
vertices[18] = [13,19]
vertices[19] = [18,20,21]
vertices[20] = [12,19,21,22,24]
vertices[21] = [19,20]
vertices[22] = [20,23]
vertices[23] = [22,24]
vertices[24] = [20,23]
vertices[25] = [26,27]
vertices[26] = [25,27]
vertices[27] = [25,26]

eberly = vertices.copy()

eg = nx.Graph(vertices)
g = nx.Graph(vertices)
nx.draw(g, coords, with_labels=True)

In [ ]:
print([i for i in nx.cycle_basis(g) if len(i) == 3])

In [ ]:
g = nx.Graph()
g.add_edges_from([(0,1), (0,2), (1,2), (0,3), (1,3), (2,3)])
nx.draw(g, with_labels=True)

In [ ]:
nx.find_cycle(g)

In [ ]:
from scipy.optimize import leastsq

e = cg[0][1]
H = e.homography
print(H)

s_pts = e.source.get_keypoint_coordinates(index=e.matches['source_idx'], homogeneous=True).values
d_pts = e.destination.get_keypoint_coordinates(index=e.matches['destination_idx'], homogeneous=True).values

#Cost Function
def f(H, ):
    pass

In [ ]:
cg[1][2].plot(clean_keys=['ransac'], line_kwargs={'linewidth':0})
show()
cg[0][1].plot(clean_keys=['ransac'], line_kwargs={'linewidth':0})
show()
cg[0][2].plot(clean_keys=['ransac'], line_kwargs={'linewidth':0})

Refinement should not look at the point to line distance, but at the error in the theoretical best, 0.

In [ ]:
e = cg[0][1]
F = e.fundamental_matrix

f_matches = e.matches[F.mask]
sc = e.source.get_keypoints().loc[f_matches['source_idx']][['x', 'y']]
dc = e.destination.get_keypoints().loc[f_matches['destination_idx']][['x', 'y']]
all_dc = e.destination.get_keypoints()[['x', 'y']]

sc_h = np.ones((len(sc), 3))
sc_h[:,:2] = sc

dc_h = np.ones((len(dc), 3))
dc_h[:,:2] = dc

for j, (i, m) in enumerate(f_matches.iterrows()):
    print(sc_h[j].T.dot(F).dot(dc_h[j]))
    print(m['source_idx'])

v = np.empty(len(dc_h))
for i, r in enumerate(dc_h):
    v[i] = sc_h[0].T.dot(F).dot(r)
print(sc)
print(min(abs(v)), np.argmin(abs(v)))
print(sc.iloc[0], dc.iloc[120])
print(f_matches[f_matches['source_idx'] == 265])

In [ ]:
cg.to_cnet(clean_keys=['suppress'])

### The graph object:
The underlying data structure is a graph, where each node is an image and each edge is the connectivity between nodes.  Nodes and Edges are classes with associated attributes and methods.  This notebook primarily focuses on the plotting functionality on the graph (and graph components).

In these notebooks, the graph object is being stored in the variable `cg`.  Access to nodes and edges is positional.

  * To access a node in the graph:  `cg[node_idx]`, e.g. `cg[0]`.
  * To access an edge in the graph: `cg[source_idx][destination_idx]`, e.g. `cg[0][1]`
  

## Plot the graph

In [ ]:
cg.plot()

## Plot features at an individual node, e.g. a single image

All defaults are used here.

In [ ]:
cg.node[1].plot()

This example specifies a plotting layout, passing in an axis object and passes along a color.  All the MatPlotLib plotting arguments are supported.

In [ ]:
ax1 = plt.subplot(1,1,1)
ax = cg.node[0].plot(ax=ax1, color='y')

## Plotting Matches on an Edge
The plotting capability on a given node is limited to a single image; one can envision the node as being the image with all associated metadata and derived information.  The edge represents the overlap between images and resultant shared information, e.g. point correspondences, a homography, etc.

#### Plot the matches between an edge using two outlier detector masks
To get a rough idea of what a 'good' results should be, we should see no, or few, lines which intersect.

In [ ]:
fig, ax = plt.subplots(1,1)
ax = cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental'], ax=ax)

#### Now plot with the added, ransac computed mask

In [ ]:
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental'], line_kwargs={'linewidth':0})

## Compute Coverage Metric
We compute a coverage metric by utilizing the homography to project the destination image corner pixel coordinates into the source image and computing the intersection.  This is a rough estimate that is as good (or poor) as the homography.

In [ ]:
H = cg.edge[0][1].homography
print('Not zero is good:', H.determinant)
print('Not huge is good: ', H.condition)
print('Shifts less than one pixel in all directions are good:', H.rmse)

In [ ]:
#Ideal coverage would be 1.0
cg.edge[0][1].coverage_ratio(clean_keys=['ransac'])

The above suggests that the quality is a function of the homography.  Just how good is the homography?  We can use the determinant (something near 1 is bad), the condition (a very large number, e.g. $10^15$ is bad), or the RMSE (reported in the x and y directions).

## Viewing Keypoint Information
Here we want to explore the attributes of the keypoints, using the masking information, e.g. the outlier detection methods.  The question is, what are the characteristics of those keypoints that have made it through the outlier detection.

In [ ]:
skp, dkp = cg.edge[0][1].keypoints(clean_keys=['ratio', 'symmetry', 'ransac'])
display(skp)
display(dkp)

## Subpixel Register
We suggest only subpixel registering 'good' candidate matches as the subpixel registration process can be time consuming.

In [ ]:
cg.edge[0][1].subpixel_register(clean_keys=['fundamental', 'symmetry', 'ratio'],template_size=5, search_size=15)

In [ ]:
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental', 'subpixel'])

In [ ]:
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental', 'subpixel'], line_kwargs={'linewidth':0})

## Apply Suppression vis Disk Covering
This method seeks to keep the $k$ strongest correlations with a good spatial distribution.

In [ ]:
cg.edge[0][1].suppress(k=100)

# Plot, in blue the points that passed all outlier detectors so far
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental', 'subpixel'], line_kwargs={'linewidth':0})
# Overlay, in red, the points that remain after suppression
cg.edge[0][1].plot(clean_keys=['suppression'], line_kwargs={'linewidth':0}, scatter_kwargs={'color':'red'})

$x^{\intercal}Fx = 0$

$d = \frac{\lvert ax_{0} + by_{0} + c \rvert}{\sqrt{a^{2} + b^{2}}}$